<a href="https://colab.research.google.com/github/Lkrasnop/Master-degree-Data_science_final_project/blob/data-preparation/Final_Project_EDA_class_data_preparation_stage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Final project - Lior Krasnopolski and Karin Shauli - class of data preparation

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
from statistics import mean
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn.cluster as cluster
from sklearn.cluster import KMeans
import numpy as np

# data preparation class

In [87]:
class data_preparation:
  def __init__(self
               # parameters of our source
               ,our_path_gene_source
               ,gene_primry_desc
               ,elias_gene_desc
               ,symbol_splitting
               #article parameters
               ,article_path_gen_source
               ,article_gene_primry_desc
               ,article_gene_elias_desc
               ,article_symbol_splitting
               #normalized parameters
               ,normalized_data_path
               #patient info
               ,patient_info_path
               #saving path for comparision function between our gene elias and article's elias data set
               ,path_to_dataframe_our_vs_article
               #saving final normalized data frame
               ,path_for_df_normalized_final):

    ###initialize parameters ########################################################
    #parameters for our gene data set
    self.our_path_gene_source = our_path_gene_source
    self.gene_primry_desc = gene_primry_desc
    self.elias_gene_desc = elias_gene_desc
    self.symbol_splitting = symbol_splitting
    #parameters for article gene data set
    self.article_gene_primry_desc = article_gene_primry_desc
    self.symbol_splitting_article = article_symbol_splitting
    self.article_path_gen_source = article_path_gen_source
    self.article_gene_elias_desc = article_gene_elias_desc
    #parameters for our normalized data set
    self.normalized_data_path = normalized_data_path
    #parameters for patient info
    self.patient_info_path = patient_info_path
    #parameters for saving path for comparision function between our gene elias and article's elias data set
    self.path_to_dataframe_our_vs_article = path_to_dataframe_our_vs_article
    #saving final normalized data frame
    self.path_for_df_normalized_final = path_for_df_normalized_final


  ###perapre our gene data set updated ##############################################
  def our_gene_desc_updated(self):
    df_our_gene_data_orginal = pd.read_csv('{}'.format(self.our_path_gene_source))
    df_our_gene_data_orginal = df_our_gene_data_orginal[['{}'.format(self.gene_primry_desc),
                                                         '{}'.format(self.elias_gene_desc)]].drop_duplicates().reset_index(drop = True)
    #split the name into two columns in order to get name column with first assignment.
    df_our_gene_data = df_our_gene_data_orginal.copy()
    df_our_gene_data['{}'.format(self.elias_gene_desc)] = df_our_gene_data['{}'.format(self.elias_gene_desc)].str.split('{}'.format(self.symbol_splitting)).str[0]

    #get our elias data
    df_our_gene_elias = df_our_gene_data_orginal['{}'.format(self.elias_gene_desc)].str.split('{}'.format(self.symbol_splitting), expand=True)

    #rename the elias columns description
    for elem in range(0,len(df_our_gene_elias.columns)):
        df_our_gene_elias.rename(columns={elem:"alias-our-dataframe {}".format(elem+1)},inplace =True)
    df_our_gene_elias = df_our_gene_elias.drop_duplicates().reset_index(drop = True)

    #clean the original data with elias primary name
    cleaned_our_data_gene_set = df_our_gene_data.loc[df_our_gene_data['{}'.format(self.elias_gene_desc)].isin(df_our_gene_elias['alias-our-dataframe 1'])].dropna()
    return df_our_gene_data ,cleaned_our_data_gene_set,  df_our_gene_elias


  ###perapre article gene data set##############################################
  def article_gene_source(self):
    # gert of list_sheet_names
    df = pd.ExcelFile('{}'.format(self.article_path_gen_source))
    list_sheet_names = df.sheet_names
    del df
    # get dataframe for gene dataset from aritcle
    data_frame_gene = []
    for name in list_sheet_names:
      temp = pd.read_excel('{}'.format(self.article_path_gen_source) , sheet_name='{}'.format(name))
      temp = temp[['{}'.format(self.article_gene_primry_desc),'{}'.format(self.article_gene_elias_desc)]]
      temp['group_name'] = name
      data_frame_gene.append(temp)

    data_frame_gene = pd.concat(data_frame_gene)
    #split the alias
    split_alias = data_frame_gene['{}'.format(self.article_gene_elias_desc)].str.split('{}'.format(self.symbol_splitting_article), expand=True)
    #change names of added columns
    for elem in range(0,len(split_alias.columns)):
      split_alias.rename(columns={elem:"alias {}".format(elem+1)},inplace =True)
    #merge alias splited dataframe into gene frame
    data_frame_gene = pd.concat([data_frame_gene, split_alias], axis=1)
    #drop alias inital column
    data_frame_gene = data_frame_gene.drop(['{}'.format(self.article_gene_elias_desc)], axis = 1)
    return data_frame_gene


  ###get normalized data for each patient##############################################
  #taking almost 3 hours to complete the execution of this function
  def genes_from_article_source_isin_genes_from_our_source(self):
    #article
    data_frame_article_gene_source = self.article_gene_source()
    data_frame_our_gene_source = self.our_gene_desc_updated()
    data_frame_our_gene_source = data_frame_our_gene_source[2]


    data = {'article': [], 'our_source': []}
    df_article_vs_our_source = pd.DataFrame(data)
    elias = data_frame_our_gene_source.fillna("None")
    acadaemic = data_frame_article_gene_source.fillna("None")

    for elem_idx in range(len(elias)):
        for elem2_idx in range(len(acadaemic)):
          our = [value for value in elias.iloc[elem_idx, :] if "None" not in str(value)]
          article = [value for value in acadaemic.iloc[elem2_idx, :] if "None" not in str(value)]
          if any(word1 in our for word1 in article):
              # add to two different columns at the same dataframe
              df_article_vs_our_source = df_article_vs_our_source.append({'our_source': elias.iloc[elem_idx, 0], 'article': acadaemic.iloc[elem2_idx, 0]}, ignore_index=True)
        #rename 'our_source' column
        df_article_vs_our_source.rename(columns={'our_source':"gene"},inplace =True)

        #remove duplicates
        df_article_vs_our_source = df_new_article.drop_duplicates().reset_index(drop =True)

        #saving the data frame
        df_article_vs_our_source.to_csv('{}'.format(self.path_to_dataframe_our_vs_article))
        return df_article_vs_our_source

  ### percentage of gene integrity compare before and after cleaning#####################
  def percentage_of_gene_integrity(self):
    article_data_before = self.article_gene_source()
    article_data_after = pd.read_csv('{}'.format(self.path_to_dataframe_our_vs_article))
    article_data_after.rename(columns={'our_source':"gene"},inplace =True)

    print('before cleaning irrelevant genes: {}'.format(article_data_before['{}'.format(self.article_gene_primry_desc)].nunique()))
    print('after cleaning irrelevant genes: {}'.format(article_data_after['article'].nunique()))
    return np.round((1-((article_data_before['{}'.format(self.article_gene_primry_desc)].nunique()- article_data_after['article'].nunique())/article_data_before['{}'.format(self.article_gene_primry_desc)].nunique()))*100,2)


  ###get normalized data for each patient################################################
  def get_normalized_data(self):
    df_normalized_celltype_reconstruct  = pd.read_csv('{}'.format(self.normalized_data_path))
    df_normalized_celltype_reconstruct = df_normalized_celltype_reconstruct.drop(columns = ['Unnamed: 0'] , axis=1)
    return df_normalized_celltype_reconstruct


  ###get pateint info data set##########################################################
  def get_patient_info(self):
    df_info = pd.read_csv('{}'.format(self.patient_info_path))
    df_info = df_info[(df_info['gender']== 'Male')]
    df_info = df_info[(df_info['age']>=20) & (df_info['age']<=80)]
    df_info = df_info.reset_index(drop = True)
    df_info.rename(columns={'GSM':"source_name"},inplace =True)
    return df_info

  #get gene and gc after cleaning the irrelevant cases###################################
  def get_final_our_gene_data_set(self):
    our_gene_after_comparison_with_article = pd.read_csv('{}'.format(self.path_to_dataframe_our_vs_article))
    our_gene_after_comparison_with_article.rename(columns={'our_source':"gene"},inplace =True)
    gene_and_cg_after_inital_cleaning = self.our_gene_desc_updated()[1]

    final_our_gene_data_set = pd.merge(gene_and_cg_after_inital_cleaning,
                                       our_gene_after_comparison_with_article,
                                       left_on = ['{}'.format(self.elias_gene_desc)],
                                       right_on = ['gene'],
                                       how = 'inner')
    final_our_gene_data_set = final_our_gene_data_set.drop(['Unnamed: 0','gene'] , axis = 1)
    final_our_gene_data_set.rename(columns={'{}'.format(self.elias_gene_desc):"our_gene"},inplace =True)
    return final_our_gene_data_set


  ###get normalized data for each patient after cleaning cgs and source name ###########
  def get_normalized_data_after_cleaning(self):
    data_frame_gene = self.get_final_our_gene_data_set()
    # data_frame_gene.rename(columns={'our_source':"Name"},inplace =True)
    df_normalized_celltype_reconstruct = self.get_normalized_data()
    df_info = self.get_patient_info()

    #filtering only the relevant cgs
    df_normalized_final = df_normalized_celltype_reconstruct.loc[df_normalized_celltype_reconstruct['source_name'].isin(df_info['source_name'])]
    df_normalized_final = df_normalized_final.loc[df_normalized_final['cg'].isin(data_frame_gene['Name'])]
    df_normalized_final = df_normalized_final.dropna()

    #merge the final normalized data frame
    df_normalized_final = pd.merge(df_normalized_final ,df_info , left_on =['source_name'] , right_on=['source_name'],how='left' )
    df_normalized_final = pd.merge(df_normalized_final ,data_frame_gene , left_on =['cg'] , right_on=['Name'],how='left' )

    #drop irrelevant columns
    df_normalized_final = df_normalized_final.drop(['disease state','gender','tissue','Name'] , axis = 1)

    #saving dataframe
    df_normalized_final.to_csv('{}'.format(self.path_for_df_normalized_final))

    return df_normalized_final



# initiate parameters

In [88]:
#our gene source - parameters
our_gene_path = '/content/drive/MyDrive/Final Project Data Science Master/קבצים ישנים/all_450K.csv'
gene_primry_desc = 'Name'
elias_gene_desc = 'UCSC_RefGene_Name'
symbol_splitting = ';'

#article - parameters
article_path_gen_source = '/content/drive/MyDrive/Final Project Data Science Master/ 41419_2021_4121_MOESM1_ESM_cleaned.xlsx'
article_gene_primry_desc = 'Official Gene symbol'
article_gene_elias_desc = 'Alias (optional)'
symbol_splitting_article = ','

#normalized data parameters
normalized_data_path = '/content/drive/MyDrive/Final Project Data Science Master/df_GSE87571_normalized_celltype_reconstruct.csv'

#patient info data set
patient_info_path = '/content/drive/MyDrive/Final Project Data Science Master/GSE87571_info.csv'


#######saving paths###################

#saving path for comparision function between our gene elias and article's elias data set
path_to_dataframe_our_vs_article = '/content/drive/MyDrive/Final Project Data Science Master/data preparation/df_article_vs_our_source.csv'

#saving path for final normalized dataframe
path_for_df_normalized_final = '/content/drive/MyDrive/Final Project Data Science Master/data preparation/df_normalized_final.csv'

data_preparation = data_preparation(our_gene_path,
                                    gene_primry_desc,
                                    elias_gene_desc,
                                    symbol_splitting,
                                    article_path_gen_source,
                                    article_gene_primry_desc,
                                    article_gene_elias_desc,
                                    symbol_splitting_article,
                                    normalized_data_path,
                                    patient_info_path,
                                    path_to_dataframe_our_vs_article,
                                    path_for_df_normalized_final)

# import data

In [74]:
data = data_preparation.get_normalized_data_after_cleaning()


<ipython-input-72-b564894d1572>:48: DtypeWarning: Columns (2,4,11,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_our_gene_data_orginal = pd.read_csv('{}'.format(self.our_path_gene_source))


In [91]:
data

,source_name,cg,normalized,age,our_gene,article
0,GSM2333902,cg00002028,0.022672,55.0,PINK1,PINK1
1,GSM2333903,cg00002028,0.020650,23.0,PINK1,PINK1
2,GSM2333901,cg00002028,0.025571,72.0,PINK1,PINK1
3,GSM2333905,cg00002028,0.035011,74.0,PINK1,PINK1
4,GSM2333908,cg00002028,0.031354,38.0,PINK1,PINK1
...,...,...,...,...,...,...
3277736,GSM2334627,cg27664530,0.070926,53.0,PIK3AP1,PIK3AP1
3277737,GSM2334630,cg27664530,0.124881,70.0,PIK3AP1,PIK3AP1
3277738,GSM2334631,cg27664530,0.111110,72.0,PIK3AP1,PIK3AP1
3277739,GSM2334632,cg27664530,0.118449,54.0,PIK3AP1,PIK3AP1


In [86]:
# genes_lst = ['ATG7', 'NBR1', 'ULK1', 'ULK2', 'WDR45']
data[data['article']=='ATG7']

,source_name,cg,normalized,age,our_gene,article
22714,GSM2333902,cg00170343,0.052571,55.0,ATG7,ATG7
22715,GSM2333903,cg00170343,0.027703,23.0,ATG7,ATG7
22716,GSM2333901,cg00170343,0.034449,72.0,ATG7,ATG7
22717,GSM2333905,cg00170343,0.050803,74.0,ATG7,ATG7
22718,GSM2333908,cg00170343,0.061796,38.0,ATG7,ATG7
...,...,...,...,...,...,...
3214026,GSM2334627,cg27083087,0.984116,53.0,VGLL4,ATG7
3214027,GSM2334630,cg27083087,0.980578,70.0,VGLL4,ATG7
3214028,GSM2334631,cg27083087,0.982991,72.0,VGLL4,ATG7
3214029,GSM2334632,cg27083087,0.980055,54.0,VGLL4,ATG7


In [92]:
data_2 = data_preparation.article_gene_source()
data_2 = data_2[['Official Gene symbol','group_name']]
data_2

<ipython-input-87-7b3f005ff7ab>:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['group_name'] = name


,Official Gene symbol,group_name
0,LAMTOR1,mTOR and upstream pathways
1,LAMTOR2,mTOR and upstream pathways
2,LAMTOR3,mTOR and upstream pathways
3,LAMTOR4,mTOR and upstream pathways
4,LAMTOR5,mTOR and upstream pathways
...,...,...
29,RAB34,Lysosome-related genes
30,RILPL1,Lysosome-related genes
31,SCPEP1,Lysosome-related genes
32,SLC38A9,Lysosome-related genes


In [95]:
data_3 = pd.merge(data , data_2 , left_on=['article'], right_on=['Official Gene symbol'],how = 'left').drop_duplicates().drop(['Official Gene symbol'],axis = 1)
data_3

,source_name,cg,normalized,age,our_gene,article,group_name
0,GSM2333902,cg00002028,0.022672,55.0,PINK1,PINK1,Autophagy core
1,GSM2333902,cg00002028,0.022672,55.0,PINK1,PINK1,Mitophagy
2,GSM2333903,cg00002028,0.020650,23.0,PINK1,PINK1,Autophagy core
3,GSM2333903,cg00002028,0.020650,23.0,PINK1,PINK1,Mitophagy
4,GSM2333901,cg00002028,0.025571,72.0,PINK1,PINK1,Autophagy core
...,...,...,...,...,...,...,...
3754730,GSM2334627,cg27664530,0.070926,53.0,PIK3AP1,PIK3AP1,Autophagy core
3754731,GSM2334630,cg27664530,0.124881,70.0,PIK3AP1,PIK3AP1,Autophagy core
3754732,GSM2334631,cg27664530,0.111110,72.0,PIK3AP1,PIK3AP1,Autophagy core
3754733,GSM2334632,cg27664530,0.118449,54.0,PIK3AP1,PIK3AP1,Autophagy core


#test

In [ ]:
%%time
df_new = []
df_new_article = []
data = {'article': [], 'our_source': []}
df_article_vs_our_source = pd.DataFrame(data)
elias = elias.fillna("None")
acadaemic = acadaemic.fillna("None")

for elem_idx in range(len(elias)):
    for elem2_idx in range(len(acadaemic)):
      our = [value for value in elias.iloc[elem_idx, :] if "None" not in str(value)]
      article = [value for value in acadaemic.iloc[elem2_idx, :] if "None" not in str(value)]
      if any(word1 in our for word1 in article):
          # add to two different columns at the same dataframe
          df_article_vs_our_source = df_article_vs_our_source.append({'our_source': elias.iloc[elem_idx, 0], 'article': acadaemic.iloc[elem2_idx, 0]}, ignore_index=True)

df_article_vs_our_source = df_article_vs_our_source.drop_duplicates().reset_index(drop =True)
display(df_article_vs_our_source)

